In [ ]:
import pandas as pd
from bandit_models import *

In [ ]:
#example list of different model names - different names (RW, HMM) and augmentations (bias, RP - will be used to pick the corresponding model class and add extra augmentations)
#possible model: random, WSLS, choice_kernel, RW, PH, HMM
#possible augmentations: RP, bias, sticky, forgetting .. - explained in models.py 

models = [ 
    'RW_RP_bias', 'RW_RP_sticky', 'RW_RP_bias_sticky',
    'HMM_RP_bias', 'HMM_RP_fixed_aur','HMM_RP_fixed_aur_bias', 
    'HMM_RP_bias_fixed_beta', 'HMM_RP_fixed_aur_fixed_beta','HMM_RP_fixed_aur_bias_fixed_beta',
    'HMM_RP_bias_sticky', 'HMM_RP_fixed_aur_sticky','HMM_RP_fixed_aur_bias_sticky', 
    'HMM_RP_bias_fixed_beta_sticky', 'HMM_RP_fixed_aur_fixed_beta_sticky','HMM_RP_fixed_aur_bias_fixed_beta_sticky'
    ]

example from models.py how different augmentations added to RW :  

        if 'RP' in model: self.RP = True #reward-punishment = different alphas for rewarded/unrewarded trials 
        if 'DSA' in model: self.DSA = True #double updating - counterfactual updating of values for unchosen option
        if 'same_alpha' in model: self.same_alpha = True #for DSA - whether use same or different alpha for counterfactual update 
        if 'scale_alpha' in model: self.scale_alpha = True #for DSA - can get alpha_c by scaling alpha 
                
        if 'forgetting' in model: self.forgetting = True 
        if 'bias' in model: self.go_bias = True
        if 'sensitivity' in model: self.sensitivity = True #scaling of reward by rho - (0-1)
        if 'sticky' in model: self.sticky = True #addition of a choice kernel 
        if 'repeat' in model: self.repeat = True #increase tendency to repeat the choice 
        
        #forgetting options 
        if 'reset_to_mean' in model: self.reset_to_mean = True #one trial reset of unchosen option to mean
        if 'reset_to_neutral' in model: self.reset_to_neutral = True #one trial reset of unchosen option to 0.5
        if 'forget_to_mean' in model: self.forget_to_mean = True #gradual decay bound at 0.5 
        if 'forget_to_neutral' in model: self.forget_to_neutral = True #gradual decay bound at mean 


same for HMM:
        if 'RP' in model: self.RP = True #reward-punishment = different alphas for rewarded/unrewarded trials 
        if 'bias' in model: self.bias = True
        if 'sticky' in model: self.sticky = True  #addition of a choice kernel 
        if 'fixed_beta' in model: self.fixed_beta = True  #version with fixed beta as suggested in quentin's paper 
        if 'fixed_gamma' in model: self.fixed_gamma = True #for the version with fixed transition matrix 
        if 'fixed_aur' in model: self.fixed_aur = True #in previous fits alpha_ur (d) often came out at the lowest value close to 0 - this version fixes alpha_ur at 0 from the start - so no updating at reward omission
        if 'repeat' in model: self.repeat = True #increase tendency to repeat the choice 


In [ ]:
#models classes and fitting
#fitting takes pandas data frame with choices in column 'right_or_left' - 0 or 1; rewards in 'reward' - 0 or 1; 
#fit each session separately; splits different conditions per mouse, if only one condition - 


fit_df = pd.DataFrame()
#based on the model name - pick the model class to run fits 
for model in models:
    if 'random' in model:
        fit = random_fit(data,model)
        temp = fit.fit_data()
        fit_df =pd.concat([fit_df,temp])
    
    if 'WSLS' in model:
        fit = WSLS_fit(data,model)
        temp = fit.fit_data()
        fit_df =pd.concat([fit_df,temp])
    
    if 'choice_kernel' in model:
        fit = CK_fit(data,model)
        temp = fit.fit_data()
        fit_df =pd.concat([fit_df,temp])

    if 'RW' in model:
        fit = RW_fit(data,model)
        temp = fit.fit_data()
        fit_df =pd.concat([fit_df,temp])
    
    if 'HMM' in model:
        fit = HMM_fit(data,model)
        temp = fit.fit_data()
        fit_df =pd.concat([fit_df,temp])

In [ ]:
#example simulations
to_sim = ['RW','RW_forgetting','HMM_RP_fixed_aur_fixed_beta','RW_DSA_same_alpha_RP_bias']

sims = pd.DataFrame()


sims = pd.DataFrame()
for model in to_sim:
    if 'RW' in model:
        #define model
        sim = RW_sim(model = model,fit_df = data_fits[data_fits.model==model],from_fit = True)#use parameters from fits 
        #run simulation 
        temp = sim.simulate(runs = 50,track_var = True) #track_var - tracks values and RPEs 
        sims =pd.concat([sims,temp]).reset_index(drop = True)
            
    if 'HMM' in model:
        sim = HMM_sim(model = model,fit_df = data_fits[data_fits.model==model],from_fit = True)
        temp = sim.simulate(runs = 50,track_var = True)
        temp['runs'] = i
        sims =pd.concat([sims,temp]).reset_index(drop = True)
        
sims['condition'] = 'sim'
